<h2>--- Day 18: Like a GIF For Your Yard ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2015FSharp/master?urlpath=lab%2Ftree%2FDay18.ipynb)

<p>After the <a href="6">million lights incident</a>, the fire code has gotten stricter: now, at most <span title="This is an outrage!  We're going to the next town hall meeting.">ten thousand lights</span> are allowed.  You arrange them in a 100x100 grid.</p>
<p>Never one to let you down, Santa again mails you instructions on the ideal lighting configuration.  With so few lights, he says, you'll have to resort to <em>animation</em>.</p>
<p>Start by setting your lights to the included initial configuration (your puzzle input).  A <code>#</code> means "on", and a <code>.</code> means "off".</p>
<p>Then, animate your grid in steps, where each step decides the next configuration based on the current one.  Each light's next state (either on or off) depends on its current state and the current states of the eight lights adjacent to it (including diagonals).  Lights on the edge of the grid might have fewer than eight neighbors; the missing ones always count as "off".</p>
<p>For example, in a simplified 6x6 grid, the light marked <code>A</code> has the neighbors numbered <code>1</code> through <code>8</code>, and the light marked <code>B</code>, which is on an edge, only has the neighbors marked <code>1</code> through <code>5</code>:</p>
<pre><code>1B5...
234...
......
..123.
..8A4.
..765.
</code></pre>
<p>The state a light should have next is based on its current state (on or off) plus the <em>number of neighbors that are on</em>:</p>
<ul>
<li>A light which is <em>on</em> stays on when <code>2</code> or <code>3</code> neighbors are on, and turns off otherwise.</li>
<li>A light which is <em>off</em> turns on if exactly <code>3</code> neighbors are on, and stays off otherwise.</li>
</ul>
<p>All of the lights update simultaneously; they all consider the same current state before moving to the next.</p>
<p>Here's a few steps from an example configuration of another 6x6 grid:</p>
<pre><code>Initial state:
.#.#.#
...##.
#....#
..#...
#.#..#
####..

After 1 step:
..##..
..##.#
...##.
......
#.....
#.##..

After 2 steps:
..###.
......
..###.
......
.#....
.#....

After 3 steps:
...#..
......
...#..
..##..
......
......

After 4 steps:
......
......
..##..
..##..
......
......
</code></pre>
<p>After <code>4</code> steps, this example has four lights on.</p>
<p>In your grid of 100x100 lights, given your initial configuration, <em>how many lights are on after 100 steps</em>?</p>

In [ ]:
let input = File.ReadAllLines @"input/18.txt"

In [ ]:
let countActiveNeighbors (x, y) state = 
    let maxX = state |> Array.head |> Array.length
    let maxY = state |> Array.length
    [|
        ( 1,  0) // right
        ( 1,  1) // bottom right
        ( 0,  1) // bottom
        (-1,  1) // bottom left
        (-1,  0) // left
        (-1, -1) // top left
        ( 0, -1) // top
        ( 1, -1) // top right
    |]
    |> Seq.map (fun (x', y') -> x + x', y + y')
    |> Seq.filter (fun (x', y') -> -1 < x' && x' < maxX && -1 < y' && y' < maxY)
    |> Seq.map (fun (x', y') -> state.[y'].[x'])
    |> Seq.filter (fun light -> light = '#')
    |> Seq.length

In [ ]:
let toggle light activeNeighbors = 
    match light with
    | '#' -> 
        match activeNeighbors with
        | 2 | 3 -> '#'
        | _ -> '.'
    | '.' -> 
        match activeNeighbors with
        | 3 -> '#'
        | _ -> '.'
    | _ -> raise <| new Exception "Invalid light configuration."

In [ ]:
let step state = 
    let maxX = state |> Array.head |> Array.length
    let maxY = state |> Array.length
    [|
        for y = 0 to maxY - 1 do 
        [|
            for x = 0 to maxX - 1 do
            toggle state.[y].[x] (countActiveNeighbors (x, y) state)
        |]
    |]

In [ ]:
let countLights state = 
    state
    |> Seq.collect (fun line -> line)
    |> Seq.filter (fun light -> light = '#')
    |> Seq.length

In [ ]:
#!time
let initialState = input |> Array.map (fun line -> line.ToCharArray())
let steps = 
    [1..100]
    |> Seq.scan (fun state stepNo -> state |> step) initialState
    |> Seq.cache

steps
|> Seq.last
|> countLights

1061

Wall time: 2390.8408ms

### Visualization

In vscode, may need to change the notebook output setting to show all the lines (100 or more).

E.g.,
```json
{
    "notebook.output.textLineLimit": 200
}
```

In [ ]:
let printState (state: char array array) = 
    state
    |> Seq.map (fun chars -> String.Join("", chars))
    |> fun lines -> String.Join(Environment.NewLine, lines)

In [ ]:
let screen = initialState |> printState |> display

steps
|> Seq.iter (fun state -> 
    System.Threading.Thread.Sleep 100
    state |> printState |> screen.Update
)

............................................................#.......................................
.........................##......................#.#.........#...............................###....
.........................##....#.................#.#.......#..#.....................................
..............................#.#...............#...#..###...#......................................
..#...........................#.#................#..#..#.#.##.....#...##............................
.#.#.......##.....##...........#.................#..#.......#.....##..##.................###........
.##........##.....##...............................#.....###..........#.............................
..........................##.......##......................#..........###..............#......#.....
.........................#..#.....#..#........##......................#..#.............#.....#.#....
..........................##.......##........#.##.....................#..#........##...#...

<h2 id="part2">--- Part Two ---</h2>

<p>You flip the instructions over; Santa goes on to point out that this is all just an implementation of <a href="https://en.wikipedia.org/wiki/Conway's_Game_of_Life">Conway's Game of Life</a>.  At least, it was, until you notice that something's wrong with the grid of lights you bought: four lights, one in each corner, are <em>stuck on</em> and can't be turned off.  The example above will actually run like this:</p>
<pre><code>Initial state:
##.#.#
...##.
#....#
..#...
#.#..#
####.#

After 1 step:
#.##.#
####.#
...##.
......
#...#.
#.####

After 2 steps:
#..#.#
#....#
.#.##.
...##.
.#..##
##.###

After 3 steps:
#...##
####.#
..##.#
......
##....
####.#

After 4 steps:
#.####
#....#
...#..
.##...
#.....
#.#..#

After 5 steps:
##.###
.##..#
.##...
.##...
#.#...
##...#
</code></pre>
<p>After <code>5</code> steps, this example now has <code>17</code> lights on.</p>
<p>In your grid of 100x100 lights, given your initial configuration, but with the four corners always in the <em>on</em> state, <em>how many lights are on after 100 steps</em>?</p>

In [ ]:
let turnOnCorners (state: char array array) = 
    let maxX = state |> Array.head |> Array.length
    let maxY = state |> Array.length
    [|
        for y = 0 to maxY - 1 do 
        [|
            for x = 0 to maxX - 1 do
            match (x, y) with
            | (0, 0) -> '#'
            | (x', 0) when x' = maxX - 1 -> '#'
            | (x', y') when x' = maxX - 1 && y' = maxY - 1 -> '#'
            | (0, y') when y' = maxY - 1 -> '#'
            | _ -> state.[y].[x]
        |]
    |]

In [ ]:
#!time
let initialState' = 
    input 
    |> Array.map (fun line -> line.ToCharArray()) 
    |> turnOnCorners

let steps' = 
    [1..100]
    |> Seq.scan (fun state stepNo -> state |> step |> turnOnCorners) initialState'
    |> Seq.cache

steps'
|> Seq.last
|> countLights

1006

Wall time: 1041.32ms

### Visualization

In [ ]:
let screen' = initialState' |> printState |> display

steps'
|> Seq.iter (fun state -> 
    System.Threading.Thread.Sleep 100
    state |> printState |> screen'.Update
)

##............................#.............................#.....................................##
#.#..........................###.................#.#.........#....................................##
.#..........................##.##................#.#.......#..#.....................................
...........................###.###..............#...#..###...#......................................
............................##.##................#..#..#.#.##.....#...##............................
.............................###.................#..#.......#.....##..##.................###........
..............................#....................#.....###..........#.............................
...........................................................#..........###..............#......#.....
...................................##.........##......................#..#.............#.....#.#....
...................................##........#.##.....................#..#........##...#...

[Prev](Day17.ipynb) | [Next](Day19.ipynb)